In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%config InlineBackend.figure_format = 'svg'

# Здесь увеличиваю размер графиков и утсанавливаю цветовую схему:

plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['image.cmap'] = 'viridis'

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('gdrive/MyDrive/data.csv', low_memory=False)

In [ ]:
df.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%config InlineBackend.figure_format = 'svg'

# Здесь увеличиваю размер графиков и утсанавливаю цветовую схему:

plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['image.cmap'] = 'viridis'

In [ ]:
# Посмотрю на данные:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332730 entries, 0 to 332729
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   InvoiceNo     332730 non-null  object 
 1   CustomerCode  332730 non-null  object 
 2   InvoiceDate   332730 non-null  object 
 3   Amount        332730 non-null  float64
dtypes: float64(1), object(3)
memory usage: 10.2+ MB


In [ ]:
df.head(3)

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47


In [ ]:
# Привожу колонки к нужному типу данных:

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df.CustomerCode = df.CustomerCode.apply(str)
df.InvoiceNo = df.InvoiceNo.apply(str)

In [ ]:
# RFM-анализ

In [ ]:
# Мне необходимо найти последниюю дату покупки:

last_date = df.InvoiceDate.max()

In [ ]:
# Формирую таблицу с данными для анализа:

rfm_data = df.groupby('CustomerCode').agg({'InvoiceDate': lambda x: (last_date - x.max()).days, # Recency:считаю сколько дней прошло с последней покупки
                                           'InvoiceNo': lambda x: len(x), # Frequency: считаю сколько покупок совершил пользователь
                                           'Amount': lambda x: x.sum()}) # Monetary (value: Общая сумма по всем заказам                                           .reset_index()

In [ ]:
# Переименовываю колонки:

rfm_data.columns = ['Recency', 'Frequency', 'MonetaryValue']

# Перезадаю индекс:

# rfm_data = rfm_data.reset_index()

int(rfm_data.reset_index()['CustomerCode'].iloc[0])

2213019

In [ ]:
# Посчитаю квантили:

quantiles = rfm_data.quantile(q=[0.25, 0.5, 0.75])
quantiles

,Recency,Frequency,MonetaryValue
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [ ]:
# Создам функцию, относящую к классу по Recency:

def RClass(value, parameter_name, quantiles_table):
  if value <= quantiles_table[parameter_name][0.25]:
    return 1
  elif value <= quantiles_table[parameter_name][0.50]:
    return 2
  elif value <= quantiles_table[parameter_name][0.75]:
    return 3
  else:
     return 4

def FMClass(value, parameter_name, quantiles_table):
  if value <= quantiles_table[parameter_name][0.25]:
    return 4
  elif value <= quantiles_table[parameter_name][0.50]:
    return 3
  elif value <= quantiles_table[parameter_name][0.75]:
    return 2
  else:
    return 1

In [ ]:
# Произвожу сегментирование:

rfm_data['R_Quantile'] = rfm_data.Recency.apply(RClass, args=('Recency', quantiles))
rfm_data['F_Quantile'] = rfm_data.Frequency.apply(FMClass, args=("Frequency", quantiles))
rfm_data['M_Quantile'] = rfm_data.MonetaryValue.apply(FMClass, args=('MonetaryValue', quantiles))
rfm_data['RFMClass'] = rfm_data.R_Quantile.map(str) \
                            + rfm_data.F_Quantile.map(str) \
                            + rfm_data.M_Quantile.map(str)

In [ ]:
rfm_data.head()

,Recency,Frequency,MonetaryValue,R_Quartile,F_Quantile,M_Quantile,R_Quantile,RFMClass
CustomerCode,,,,,,,,
02213019,19,1,1609.20,4,4,3,4,443
02213042,22,3,9685.48,4,2,1,4,421
02213071,29,1,415.00,4,4,4,4,444
02213088,23,1,305.00,4,4,4,4,444
02213092,25,1,1412.88,4,4,3,4,443


In [ ]:
rfm_data.head()

,Recency,Frequency,MonetaryValue,R_Quantile,F_Quantile,M_Quantile,RFMClass
CustomerCode,,,,,,,
02213019,19,1,1609.20,4,4,3,443
02213042,22,3,9685.48,4,2,1,421
02213071,29,1,415.00,4,4,4,444
02213088,23,1,305.00,4,4,4,444
02213092,25,1,1412.88,4,4,3,443


In [ ]:
#### Выводы:

In [ ]:
# Максимальное число покупок, совершенных одним пользователем:
rfm_data.Frequency.max()

204

In [ ]:
# Меня интерсует верхняя граница суммы покупок пользователей с клвссом 4 подсегмента М (покупатели, которые покупают на самые маленькие суммы от 0 до 25 квантиля):
quantiles.MonetaryValue.loc[0.25]

765.0

In [ ]:
# Какова нижняя граница у количества покупок пользователй с классом 1 в подсегменте F (класс, в котором покупают часто и на большие суммы):
# Поскольку значение 3 квантиля включает в себя это число, а количество покупок число целое, то считаю так:

quantiles.Frequency.loc[0.75] + 1



4.0

In [ ]:
# Посмотрю верхнюю границу у количества покупок пользователей с классом 2 в подсегменте R:
quantiles.Recency.loc[0.25]

2.0

In [ ]:
# Какова верхняя граница дней, прошедших с момента последней покупки у пользователей с классом 2 в подсегменте R:
quantiles.Recency.loc[0.50]

8.0

In [ ]:
# Посчитаю сколько пользователей попало в топовый сегмент 111:
rfm_data.query('RFMClass == "111"').Frequency.count()

9705

In [ ]:
# Посмотрю в каком RFM сегменте у меня наибольшее количество пользователей:

rfm_data.groupby('RFMClass').size().sort_values(ascending=False).head(1)

RFMClass
444    10624
dtype: int64

In [ ]:
# Найду количество  пользователей в самом малочисленном сегменте:
rfm_data.groupby('RFMClass').size().sort_values(ascending=False).tail(1).item()

2